In [2]:
import pandas as pd
import json
import numpy as np

<h2>CARGA DE DATOS</h2>
Datos Originales:   <a href="url">https://drive.google.com/drive/folders/1TI-SsMnZsNP6t930olEEWbBQdo_yuIZF</a>

In [3]:
#Carga Dataset original business.pkl 
df_business = pd.read_pickle(r'C:\HENRY\PROYECTOS HENRY 11\PROYECTO GRUPAL\Datasets\business.pkl')

In [4]:
#Carga Dataset original checkin.json
df_checkin = pd.read_json(r'C:\HENRY\PROYECTOS HENRY 11\PROYECTO GRUPAL\Datasets\checkin.json', lines=True)

In [5]:
#Carga Dataset original review.json
chunk_size = 1000  # Número de registros a leer por cada chunk
chunks = pd.read_json(r'C:\HENRY\PROYECTOS HENRY 11\PROYECTO GRUPAL\Datasets\review.json', lines=True, chunksize=chunk_size)

df_review = pd.concat(chunks, ignore_index=True)

In [6]:
#Carga Dataset original tip.json
df_tip = pd.read_json(r'C:\HENRY\PROYECTOS HENRY 11\PROYECTO GRUPAL\Datasets\tip.json', lines=True)

In [7]:
#Carga Dataset original user.parquet
df_user = pd.read_parquet(r'C:\HENRY\PROYECTOS HENRY 11\PROYECTO GRUPAL\Datasets\user.parquet')

<h2>BUSINESS</h2>

In [8]:
#Conteo de datos
len(df_business)

150346

In [9]:
#Mostrar primeros 5 registros
df_business.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,NaN,93101,34.426679,-119.711197,5.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,NaN,63123,38.551126,-90.335695,3.0,15,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,NaN,85711,32.223236,-110.880452,3.5,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,CA,19107,39.955505,-75.155564,4.0,80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,MO,18054,40.338183,-75.471659,4.5,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#Conteo de columnas duplicadas
df_business.columns.duplicated().sum()

np.int64(14)

In [11]:
len(df_business.columns)

28

In [12]:
# Eliminar columnas duplicadas 
df_business = df_business.loc[:, ~df_business.columns.duplicated()]

In [13]:
df_business.columns

Index(['business_id', 'name', 'address', 'city', 'state', 'postal_code',
       'latitude', 'longitude', 'stars', 'review_count', 'is_open',
       'attributes', 'categories', 'hours'],
      dtype='object')

In [14]:
# Buscar múltiples palabras clave ('Resort', 'bar', 'hotel'...) 
keywords = 'Resort|bar|hotel|sport|turism|gym|theater|pubs' 
df_ocios = df_business[df_business['categories'].str.contains(keywords, case=False, na=False)] 
# Verificar los resultados 
len(df_ocios)

27413

In [ ]:
#Confirmar duplicados
#  Convertir los valores de tipo dict en columnas a cadenas JSON
for column in df_ocios.columns:
    if df_ocios[column].apply(lambda x: isinstance(x, dict)).any():
        df_ocios[column] = df_ocios[column].apply(json.dumps)

# Verificar duplicados
duplicated_rows = df_ocios.duplicated()

# Mostrar cuántos duplicados hay
print(f"Total de duplicados: {duplicated_rows.sum()}")

# Eliminar duplicados, conservando el primero (opcional)
df_ocios = df_ocios.drop_duplicates()

# Verificar el DataFrame resultante
print(len(df_ocios))


C:\Users\diani\AppData\Local\Temp\ipykernel_8048\1744437901.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ocios[column] = df_ocios[column].apply(json.dumps)


Total de duplicados: 0
27413


In [16]:
# Filtrar DataFrame para categorías que contienen la palabra 'Restaurant' Y por 'food'
df_restaurants = df_business[df_business['categories'].str.contains('restaurant', case=False, na=False)]
df_restaurants = df_business[df_business['categories'].str.contains('food', case=False, na=False)]

prueba = pd.DataFrame({
    'name': df_restaurants['name'],
    'attributes': df_restaurants['attributes'],
    'categories': df_restaurants['categories']
})

# Verificar los resultados
prueba.head(12)


,name,attributes,categories
3,St Honore Pastries,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
4,Perkiomen Valley Brewery,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food"
5,Sonic Drive-In,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
9,Sonic Drive-In,"{'RestaurantsAttire': ''casual'', 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
11,Vietnamese Food Truck,"{'Alcohol': ''none'', 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks"
14,Zio's Italian Market,"{'OutdoorSeating': 'False', 'RestaurantsGoodFo...","Food, Delis, Italian, Bakeries, Restaurants"
20,Roast Coffeehouse and Wine Bar,"{'OutdoorSeating': 'False', 'Caters': 'True', ...","Coffee & Tea, Food, Cafes, Bars, Wine Bars, Re..."
27,The Green Pheasant,"{'RestaurantsGoodForGroups': 'True', 'HappyHou...","Restaurants, Japanese, Seafood"
35,Craft Hall,"{'OutdoorSeating': 'True', 'RestaurantsPriceRa...","Eatertainment, Arts & Entertainment, Brewpubs,..."
41,Tony's Restaurant & 3rd Street Cafe,"{'RestaurantsReservations': 'True', 'Restauran...","Restaurants, Specialty Food, Steakhouses, Food..."


In [17]:
#Concatenar resultados de filtro de palabras clave con resultados de filtro restaurant y food
df_ocio = pd.concat([df_restaurants, df_ocios])

In [18]:
#Exportar Datos
#df_ocio.to_json('df_businessX.json', orient='records', lines=True)

Calcular valores duplicados y eliminarlos

In [19]:
# Convertir los valores de tipo dict en columnas a cadenas JSON
for column in df_ocios.columns:
    if df_ocio[column].apply(lambda x: isinstance(x, dict)).any():
        df_ocio[column] = df_ocio[column].apply(json.dumps)

# Verificar duplicados
duplicated_rows = df_ocio.duplicated()

# Mostrar cuántos duplicados hay
print(f"Total de duplicados: {duplicated_rows.sum()}")

# Eliminar duplicados
df_ocio = df_ocio.drop_duplicates()

# Verificar el DataFrame resultante
print(len(df_ocio))

Total de duplicados: 0
63332


In [20]:
df_ocio.describe()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
count,63332,63332,63332,63332,63332,63332,63332.000000,63332.000000,63332.0,63332,63332,63332,63332,63332
unique,57163,40108,48319,986,20,2104,53989.000000,53332.000000,9.0,1078,2,54439,37942,28608
top,7Cx7EYUGQ2_KHS80s87rbw,Starbucks,,Philadelphia,PA,70130,34.420334,-119.710749,4.0,5,1,"""null""","Food, Coffee & Tea",null
freq,2,727,1158,6447,14379,963,45.000000,45.000000,14659.0,3806,47176,1916,508,4393


<h2>CHECKIN</h2>

In [21]:
df_checkin.columns

Index(['business_id', 'date'], dtype='object')

In [22]:
len(df_checkin)

131930

In [23]:
df_checkin.head(2)

,business_id,date
0,---kPU91CF4Lq2-WlRu9Lw,"2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020..."
1,--0iUa4sNDFiZFrAdIWhZQ,"2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011..."


In [24]:
#Depurar
# Filtrar df_checkin solo con IDs presentes en df_ocio
df_checkinX = df_checkin[df_checkin['business_id'].isin(df_ocio['business_id'])]
# Verificar los resultados 
print(df_checkinX)

                   business_id  \
0       ---kPU91CF4Lq2-WlRu9Lw   
1       --0iUa4sNDFiZFrAdIWhZQ   
5       --8IbOsAAxjKRoYsBFL-PA   
7       --ARBQr1WMsTWiwOKOj-FQ   
11      --MbOh2O1pATkXa7xbU6LA   
...                        ...   
131917  zzXRdzrVhfNWPHD2MeyWeA   
131918  zzbZtgPYZS8sTIWQH6DwEw   
131922  zziDpuuJw-Km1J4BaGpBKA   
131923  zzjCxn89a7RQo8keIOO_Ag   
131925  zznJox6-nmXlGYNWgTDwQQ   

                                                     date  
0       2020-03-13 21:10:56, 2020-06-02 22:18:06, 2020...  
1       2010-09-13 21:43:09, 2011-05-04 23:08:15, 2011...  
5       2015-06-06 01:03:19, 2015-07-29 16:50:58, 2015...  
7       2014-12-12 00:44:23, 2015-01-09 00:19:52, 2015...  
11      2013-04-21 01:52:06, 2013-05-02 04:02:07, 2013...  
...                                                   ...  
131917  2019-06-28 23:00:57, 2019-07-12 23:55:18, 2019...  
131918  2010-08-13 07:51:27, 2010-08-14 06:35:15, 2010...  
131922  2011-04-23 16:56:26, 2012-12-18 19:49:13, 2

In [25]:
df_checkinX.duplicated().sum()

np.int64(0)

In [26]:
#Exportar Datos
#df_checkinX.to_parquet('df_checkinX.parquet', engine='pyarrow')

<h2>REVIEW</h2>

In [27]:
df_review.head(2)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18


In [28]:
df_review.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [29]:
len(df_review)

6990280

In [30]:
df_review.isnull().sum()

review_id      0
user_id        0
business_id    0
stars          0
useful         0
funny          0
cool           0
text           0
date           0
dtype: int64

In [31]:
df_review.duplicated().sum()

np.int64(0)

In [32]:
#Depurar
# Combinar los DataFrames basados en business_id 
df_reviewX = df_review[df_review['business_id'].isin(df_ocio['business_id'])] 
# Verificar los resultados 
print(df_reviewX.head())

                review_id                 user_id             business_id  \
0  KU_O5udG6zpxOg-VcAEodg  mh_-eMZ6K5RLWhZyISBhwA  XQfwVwDr-v0ZS3_CbbE5Xw   
1  BiTunyQ73aT9WBnpR9DZGw  OyoGAe7OKpv6SyGZT5g77Q  7ATYjTIgM3jUlt4UM3IypQ   
4  Sx8TMOWLNuJBWer-0pcmoA  bcjbaE6dDog4jkNY91ncLQ  e4Vwtrqf-wpJfwesgvdgxQ   
5  JrIxlS1TzJ-iCu79ul40cQ  eUta8W_HdHMXPzLBBZhL1A  04UD14gamNjLY0IDYVhHJg   
6  6AxgBCNX_PNTOxmbRSwcKQ  r3zeYsv1XFBRA4dJpL78cw  gmjsEdUsKpj9Xxu6pdjH0g   

   stars  useful  funny  cool  \
0      3       0      0     0   
1      5       1      0     1   
4      4       1      0     1   
5      1       1      2     1   
6      5       0      2     0   

                                                text                date  
0  If you decide to eat here, just be aware it is... 2018-07-07 22:09:11  
1  I've taken a lot of spin classes over the year... 2012-01-03 15:28:18  
4  Cute interior and owner (?) gave us tour of up... 2017-01-14 20:54:15  
5  I am a long term frequent customer 

In [33]:
len(df_reviewX)

4067218

In [ ]:
#Exportar Datos
#df_reviewX.to_parquet('df_reviewX.parquet', engine='pyarrow')

<h2>TIP</h2>

In [35]:
df_tip.columns

Index(['user_id', 'business_id', 'text', 'date', 'compliment_count'], dtype='object')

In [36]:
len(df_tip)

908915

In [37]:
df_tip.head()


,user_id,business_id,text,date,compliment_count
0,AGNUgVwnZUey3gcPCJ76iw,3uLgwr0qeCNMjKenHJwPGQ,Avengers time with the ladies.,2012-05-18 02:17:21,0
1,NBN4MgHP9D3cw--SnauTkA,QoezRbYQncpRqyrLH6Iqjg,They have lots of good deserts and tasty cuban...,2013-02-05 18:35:10,0
2,-copOvldyKh1qr-vzkDEvw,MYoRNLb5chwjQe3c_k37Gg,It's open even when you think it isn't,2013-08-18 00:56:08,0
3,FjMQVZjSqY8syIO-53KFKw,hV-bABTK-glh5wj31ps_Jw,Very decent fried chicken,2017-06-27 23:05:38,0
4,ld0AperBXk1h6UbqmM80zw,_uN0OudeJ3Zl_tf6nxg5ww,Appetizers.. platter special for lunch,2012-10-06 19:43:09,0


In [38]:
#Depurar
# Combinar los DataFrames basados en business_id 
df_tipX = df_tip[df_tip['business_id'].isin(df_ocio['business_id'])] 
# Verificar los resultados 
print(df_tipX.head())

                  user_id             business_id  \
1  NBN4MgHP9D3cw--SnauTkA  QoezRbYQncpRqyrLH6Iqjg   
2  -copOvldyKh1qr-vzkDEvw  MYoRNLb5chwjQe3c_k37Gg   
5  trf3Qcz8qvCDKXiTgjUcEg  7Rm9Ba50bw23KTA8RedZYg   
6  SMGAlRjyfuYu-c-22zIyOg  kH-0iXqkL7b8UXNpguBMKg   
7  YVBB9g23nuVJ0u44zK0pSA  jtri188kuhe_AuEOJ51U_A   

                                                text                date  \
1  They have lots of good deserts and tasty cuban... 2013-02-05 18:35:10   
2             It's open even when you think it isn't 2013-08-18 00:56:08   
5  Chili Cup + Single Cheeseburger with onion, pi... 2012-03-13 04:00:52   
6  Saturday, Dec 7th 2013, ride Patco's Silver Sl... 2013-12-03 23:42:15   
7  This is probably the best place in the cool Sp... 2016-11-22 22:14:58   

   compliment_count  
1                 0  
2                 0  
5                 0  
6                 0  
7                 0  


In [39]:
len(df_tipX)

568622

In [40]:
#Exportar Datos
df_tipX.to_parquet('df_tipX.parquet', engine='pyarrow')

<h2>USER</h2>

In [41]:
df_user.columns

Index(['user_id', 'name', 'review_count', 'yelping_since', 'useful', 'funny',
       'cool', 'elite', 'friends', 'fans', 'average_stars', 'compliment_hot',
       'compliment_more', 'compliment_profile', 'compliment_cute',
       'compliment_list', 'compliment_note', 'compliment_plain',
       'compliment_cool', 'compliment_funny', 'compliment_writer',
       'compliment_photos'],
      dtype='object')

In [42]:
df_user.head(5)

,user_id,name,review_count,yelping_since,useful,funny,cool,elite,friends,fans,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,qVc8ODYU5SZjKXVBgXdI7w,Walker,585,2007-01-25 16:47:26,7217,1259,5994,2007,"NSCy54eWehBJyZdG2iE84w, pe42u7DcCH2QmI81NX-8qA...",267,...,65,55,56,18,232,844,467,467,239,180
1,j14WgRoU_-2ZE1aw1dXrJg,Daniel,4333,2009-01-25 04:35:42,43091,13066,27281,"2009,2010,2011,2012,2013,2014,2015,2016,2017,2...","ueRPE0CX75ePGMqOFVj6IQ, 52oH4DrRvzzl8wh5UXyU0A...",3138,...,264,184,157,251,1847,7054,3131,3131,1521,1946
2,2WnXYQFK0hXEoTxPtV2zvg,Steph,665,2008-07-25 10:41:00,2086,1010,1003,"2009,2010,2011,2012,2013","LuO3Bn4f3rlhyHIaNfTlnA, j9B4XdHUhDfTKVecyWQgyA...",52,...,13,10,17,3,66,96,119,119,35,18
3,SZDeASXq7o05mMNLshsdIA,Gwen,224,2005-11-29 04:38:33,512,330,299,"2009,2010,2011","enx1vVPnfdNUdPho6PH_wg, 4wOcvMLtU6a9Lslggq74Vg...",28,...,4,1,6,2,12,16,26,26,10,9
4,hA5lMy-EnncsH4JoR-hFGQ,Karen,79,2007-01-05 19:40:59,29,15,7,,"PBK4q9KEEBHhFvSXCUirIw, 3FWPpM7KU1gXeOM_ZbYMbA...",1,...,1,0,0,0,1,1,0,0,0,0


In [43]:
df_user.isnull().sum()

user_id               0
name                  0
review_count          0
yelping_since         0
useful                0
funny                 0
cool                  0
elite                 0
friends               0
fans                  0
average_stars         0
compliment_hot        0
compliment_more       0
compliment_profile    0
compliment_cute       0
compliment_list       0
compliment_note       0
compliment_plain      0
compliment_cool       0
compliment_funny      0
compliment_writer     0
compliment_photos     0
dtype: int64

In [44]:
df_user.duplicated().sum()

np.int64(117700)

In [45]:
len(df_user)

2105597

In [46]:
#Eliminar los duplicados
df_user.drop_duplicates(inplace=True)
len(df_user)

1987897

In [47]:
df_reviewX.columns

Index(['review_id', 'user_id', 'business_id', 'stars', 'useful', 'funny',
       'cool', 'text', 'date'],
      dtype='object')

In [48]:
#Depurar
# Combinar los DataFrames basados en business_id 
df_userX = df_user[df_user['user_id'].isin(df_reviewX['user_id'])] 
# Verificar los resultados 
print(df_userX.head())

                  user_id    name  review_count        yelping_since  useful  \
0  qVc8ODYU5SZjKXVBgXdI7w  Walker           585  2007-01-25 16:47:26    7217   
1  j14WgRoU_-2ZE1aw1dXrJg  Daniel          4333  2009-01-25 04:35:42   43091   
2  2WnXYQFK0hXEoTxPtV2zvg   Steph           665  2008-07-25 10:41:00    2086   
3  SZDeASXq7o05mMNLshsdIA    Gwen           224  2005-11-29 04:38:33     512   
4  hA5lMy-EnncsH4JoR-hFGQ   Karen            79  2007-01-05 19:40:59      29   

   funny   cool                                              elite  \
0   1259   5994                                               2007   
1  13066  27281  2009,2010,2011,2012,2013,2014,2015,2016,2017,2...   
2   1010   1003                           2009,2010,2011,2012,2013   
3    330    299                                     2009,2010,2011   
4     15      7                                                      

                                             friends  fans  ...  \
0  NSCy54eWehBJyZdG2iE84w, pe42

In [49]:
df_userX.duplicated().sum()

np.int64(0)

In [50]:
len(df_userX)

1373670

In [ ]:
#Exportar Datos
#df_userX.to_parquet('df_userX.parquet', engine='pyarrow')